# Adding Interactivity

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 12)

In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.plotting import ColumnDataSource
from bokeh.layouts import row, column, gridplot
from bokeh.transform import cumsum
output_notebook()

Loading BokehJS ...

## Hover Tool

The Hover Tool in Bokeh displays more information in a popup whenever the user hovers over a glyph.

In [3]:
from bokeh.models import HoverTool

### Plotting Points

The data

In [4]:
x1 = [1, 2, 3, 4, 5, 6]
y1 = [3, 4, 2, 3, 6, 5]

Defining an empty figure

In [5]:
p = figure(title="Hover over Points", 
    x_axis_label='x', y_axis_label='y',  y_range=(0, 7),    
    tools=[HoverTool()], tooltips="(@x,@y)",    
    width=600, height=400
)

Assigning the data and visualizing it

In [6]:
p.circle(x1, y1, size=10)
p.line(x1, y1, line_width=2)
show(p);

### Pie Chart

In [7]:
from math import pi

In [8]:
fruits = ['Guava','Mango','Orange','Lime']
counts = [2, 4, 5, 3]
angles = [counts[i]/np.sum(counts) * 2*pi for i in range(len(counts))]
colors = ['hotpink', 'gold', 'orange', 'limegreen']    

In [9]:
data={
    'fruits': fruits,
    'counts': counts,    
    'angles': angles,
    'colors': colors
}

In [10]:
source_fruit = ColumnDataSource(data = data)

In [11]:
p2 = figure(
    tools=[HoverTool()], tooltips="(@fruits,@counts)",  
    width=500, height=500
    )
p2.axis.visible = False
p2.grid.grid_line_color = None

In [12]:
p2.wedge(x=2, y=0, radius=0.8,
        start_angle=cumsum('angles', include_zero=True),
        end_angle=cumsum('angles'),
        line_color="white",
        fill_color='colors',
        legend_field='fruits',
        source=source_fruit)
p2.legend.orientation = "horizontal"
p2.legend.location = "top"        
show(p2) 

### Sunspots Dataset

Sunspots are impermanent phenomena on the Sun's photosphere that appear darker than the surrounding areas. Sunspots occur in pairs of opposite magnetic polarities. Their number varies according to the **`11-year`** solar cycle.


Source: https://en.wikipedia.org/wiki/Sunspot

Database from SIDC - Solar Influences Data Analysis Center - the solar physics research department of the Royal Observatory of Belgium. SIDC website

In [13]:
df = pd.read_csv('Sunspots.csv', index_col=0)
df.head()

,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


In [14]:
df.rename(columns = {'Monthly Mean Total Sunspot Number':'Sunspots'}, inplace=True)
df.head()

,Date,Sunspots
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


In [15]:
df.dtypes

Date         object
Sunspots    float64
dtype: object

In [16]:
df.Date = pd.to_datetime(df.Date)
df.dtypes

Date        datetime64[ns]
Sunspots           float64
dtype: object

In [17]:
df['Year']  = df.Date.dt.strftime('%Y').astype('int') 
df['Month'] = df.Date.dt.strftime('%m').astype('int') 
df['Day']   = df.Date.dt.strftime('%d').astype('int') 
df.head() 

,Date,Sunspots,Year,Month,Day
0,1749-01-31,96.7,1749,1,31
1,1749-02-28,104.3,1749,2,28
2,1749-03-31,116.7,1749,3,31
3,1749-04-30,92.8,1749,4,30
4,1749-05-31,141.7,1749,5,31


Create a line plot of the sunspots vs. time

In [18]:
p2 = figure(title="Sunspots", x_axis_label='Date', 
        y_axis_label='Monthly Mean Total Sunspot Number',
        height=400, width=900
)

In [19]:
p2.line(x=df.Date.dt.strftime('%Y'), y=df.Sunspots, line_width=2)
show(p2)

Let's add a hover tool to the graph.

In [20]:
source = ColumnDataSource(data=dict(
    x=df.Year,          # We need x and y for plotting the graph
    y=df.Sunspots,
    m=df.Month,         # We want to show month-day-year when hover the mouse over the graph
    d=df.Day)
)

Defining the `hover` tool

In [21]:
TOOLTIPS = [
    ("    Date = ", "@m-@d-@x"),
    ("Sunspots = ", "@y{0.0}"),
]

In [22]:
p2 = figure(title="Sunspots - Mouse over graph", 
        x_axis_label='Date', 
        y_axis_label='Monthly Mean Total Sunspot Number',
        tooltips=TOOLTIPS, 
        height=400, width=1000
)

In [23]:
p2.line(line_width=2, source=source)
show(p2)

You can see the date and sunspot value for every point in the graph.

### Adding box annotations

In [24]:
from bokeh.models import BoxAnnotation

In [25]:
low_box  = BoxAnnotation(top=100,    fill_alpha=0.2, fill_color='palegreen')
high_box = BoxAnnotation(bottom=300, fill_alpha=0.2, fill_color='lightcoral')

In [26]:
p2.add_layout(low_box)
p2.add_layout(high_box)
show(p2)

## Linked Brushing

Brush over any graph and the point selection reflects in all charts.

### Automobile Dataset

We will use the Automobile Data Set [https://archive.ics.uci.edu/ml/datasets/automobile] from the UCI Machine Learning Repository [https://archive-beta.ics.uci.edu/]. It includes categorical and continuous variables. 

In [27]:
# Defining the headers
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration", "num_doors", 
           "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", 
            "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", 
            "highway_mpg", "price"]

In [28]:
dfa = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data",
                  header=None, names=headers, na_values="?" )
dfa.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,...,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,...,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,...,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,...,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,...,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,...,8.0,115.0,5500.0,18,22,17450.0


### Brush tooltip

We will start by creating a common column data source for all the plots.

In [29]:
sourceA = ColumnDataSource(data=dict(
    Price  = dfa.price, 
    Hmpg   = dfa.highway_mpg,
    Cmpg   = dfa.city_mpg,
    Hpower = dfa.horsepower)
    )

In [30]:
TOOLS = "box_select, lasso_select, reset, help"

Creating a plot for `Price` vs. `Hmpg`

In [31]:
# Creating a plot for Price vs Hmpg
p1 = figure(tools = TOOLS, y_range=[0, 50000], title='Car Price vs. Highway (mpg)')
p1.circle('Hmpg', 'Price', source=sourceA, color='orangered', size=8);

Creating a plot for `Price` vs. `Cmpg`

In [32]:
# Creating a plot for Price vs Cmpg
p2 = figure(tools = TOOLS, y_range=[0, 50000], title='Car Price vs. City (mpg)')
p2.circle('Cmpg', 'Price', source=sourceA, color='orange', size=8);

Creating a plot for `Price` vs. `Hpower`

In [33]:
# Creating a plot for Price vs Hpower
p3 = figure(tools = TOOLS, y_range=[0, 50000], title='Car Price vs. Horsepower')
p3.circle('Hpower', 'Price', source=sourceA, color='steelblue', size=8);

Creating a grid

In [34]:
grid1 = gridplot([[p1, p2, p3]], width=350, height=300)
show(grid1)

Using the layout `row`

In [35]:
plot_r = row([p1,p2,p3])
show(plot_r)

In [36]:
grid2 = gridplot([[p1, p2], [p3]], width=350, height=300)
show(grid2)

In [37]:
grid3 = gridplot([[p1, p2], [None, p3]], width=350, height=300)
show(grid3)

In [38]:
grid4 = gridplot([[p1], [p2], [p3]], width=350, height=200)
show(grid4)

Using the layout `column`

In [39]:
plot_c = column([p1,p2,p3])
show(plot_c)

## References

- https://bokeh.org/
- https://docs.bokeh.org/en/latest/docs/reference/layouts.html